# 元编程

## `__new__`

我们通常把 `__init__` 称为构造方法，其实，用于构建实例的是特殊方法 `__new__` ：这是个类方法，由于使用了特殊方式处理，因此不必使用 @classmethod 装饰器，该方法必须返回一个实例。返回的实例会作为第一个参数 (即 self) 传给 `__init__` 方法。

因为调用 `__init__` 方法时要传入实例，而且禁止返回任何值，所以 `__init__` 其实称为**初始化**方法更为合适。真正的构造方法是 `__new__` 。几乎不需要自己编写 `__new__` 方法，因为从 object 类继承的实现已经足够了。

<b style="color: red">注意：</b>`__new__` 方法也可以返回其他类的实例，此时，解释器不会调用 `__init__` 方法。

Python 构建对象的过程可以用下述伪代码概括：

In [1]:
def object_maker(the_class, some_arg):
    new_object = the_class.__new__(some_arg)
    if isinstance(new_object, the_class):
        the_class.__init__(new_object, some_arg)
    return new_object

# 下述两个语句作用等效
# x = Foo('bar')
# x = object_maker(Foo, 'bar')

在 `__new__` 方法中调用 `super().__new__(cls)` 会调用 `object.__new__(cls)` ，而 object 类构建的实例其实是 cls 实例，即实例的 `__class__` 属性存储的是 cls 类的引用。(真正的构建操作由解释器调用 C 语言实现的 `object.__new__` 方法执行)

## 描述符

### 属性查找

#### 从类中查找属性

使用 `C.name` 引用类对象 C 的一个属性时，查询操作如下：

  1. 当 name 是 `C.__dict__` 中的一个键时， `C.name` 将从 `C.__dict__['name']` 中提取值 v。如果 v 是一个描述器，则 `C.name` 的值就是 `type(v).__get__(v, None, C)` ，否则，`C.name` 的值为 v
  2. 否则，`C.name` 将委托查找 C 的基类
  3. 否则引发 AttributeError

#### 从实例中查找属性

<b style="color: red">
`obj.attr` 这样的表达式不会从 obj 开始寻找 attr ，而是从 `obj.__class__` 开始，仅当类中没有名为 attr 的描述符时，才会在 obj 实例中寻找。
</b>

使用 `x.name` 引用类 C 的实例 x 的一个属性时，查询操作如下：

  1. 当 name 作为一个覆盖描述器 v 的名称在类 C （或 C 的某个祖先类）中被找到，`x.name` 的值就是 `type(v).__get__(v, x, C)`
  2. 否则，当 name 是 `x.__dict__` 中的一个键时，返回 `x.__dict__['name']`
  3. 否则，x.name 将委托查找 x 的类，即查找 `C.name`
  4. 如果 C 定义或继承了特殊方法 `__getattr__` ，则调用 `C.__getattr__(x, 'name')` ，而不是引发 AttributeError，然后根据 `__getattr__` 返回一个合适的值或者引发 AttributeError

### 覆盖型描述符 

也叫作数据描述符或强制描述符。

实现 `__set__` 方法的描述符属于覆盖型描述符，虽然描述符是类属性，但是实现了 `__set__` 方法的话，会覆盖对实例属性的赋值操作。

内置的 property 类创建的其实是覆盖型描述符，`__set__` 方法和 `__get__` 方法都实现了，`__set__` 方法默认抛出 AttributeError 异常。

#### 没有 `__get__` 方法的覆盖型描述符

通过实例读取描述符会返回描述符对象本身。如果直接通过实例的 `__dict__` 属性创建同名实例属性，以后再设置那个属性时，仍会由 `__set__` 方法接管，但是读取那个属性时，会直接从实例中返回新赋的值，而不会返回描述符对象。也就是说，实例属性会遮盖描述符，不过只有读操作如此。

### 非覆盖型描述符

也叫作非数据描述符或遮盖型描述符。

没有实现 `__set__` 方法的描述符是非覆盖型描述符。如果设置了同名的实例属性，描述符会被覆盖，致使描述符无法处理那个实例的那个属性。<b style="color: red">方法是以非覆盖型描述符实现的 (只有 `__get__` 方法) 。</b>

非覆盖型描述符可以用来实现缓存，执行某些耗费资源的计算，然后为实例设置同名属性，缓存结果。同名属性会遮盖描述符，因此后续访问会直接从实例的 `__dict__` 属性中获取值，不会触发描述符的 `__get__` 方法。

## 元类

### 使用 type 动态创建类


In [2]:
def fn(self, name='world'):
    print('Hello, %s.' % name)

Hello = type('Hello', (object,), dict(hello=fn))

h = Hello()
type(Hello), type(h)

(type, __main__.Hello)

### 使用 metaclass 控制类的创建

所有类都是 type 的实例，元类是 type 的子类，可以作为制造类的工厂。
具体来说，元类可以通过实现 `__init__` 方法定制实例。元类的 `__init__` 方法可以做到类装饰器能做的任何事情。
(如果想进一步定制类，可以在元类中实现 `__new__` 方法。不过，通常情况下实现 `__init__` 方法就够了)

In [3]:
class MetaFoo(type):
    
    def __new__(metacls, name, bases, attrs):
        print("metacls: {}, name: {}, bases: {}, attrs: {}".format(metacls, name, bases, attrs))
        return type.__new__(metacls, name, bases, attrs)

    def __init__(cls, name, bases, attrs): # name, bases, attrs: 与构建类时传给 type 的参数一样
        print("cls: {}, name: {}, bases: {}, attrs: {}".format(cls, name, bases, attrs))

class Foo(str, metaclass=MetaFoo):
    pass

print(dir(Foo))

metacls: <class '__main__.MetaFoo'>, name: Foo, bases: (<class 'str'>,), attrs: {'__qualname__': 'Foo', '__module__': '__main__'}
cls: <class '__main__.Foo'>, name: Foo, bases: (<class 'str'>,), attrs: {'__qualname__': 'Foo', '__module__': '__main__'}
['__add__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'ma